In [8]:
import csv
import json
from pymongo import MongoClient
# pprint library is used to make the output look more pretty
from pprint import pprint
import numpy as np
import time
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

# Loading on Mongo

In [9]:
# Connect to Mongo
client = MongoClient('localhost', 27017)

db = client["nyc_pymongo"]

In [10]:
# storing collections as python variables

taxis_coll = db["nyc_one"]
lookup_coll = db["lookup"]

### Importing data

In [4]:
# loading data on mongo collections

start = time.time()
with open('nyc_final.json', 'r') as f:
    file_data = json.load(f)

taxis_coll.insert_many(file_data)

end = time.time()
print(round((end - start)/60 ,2))

15.78


In [5]:
# loading data on mongo collections

start = time.time()
with open('lookup.json', 'r') as f:
    file_data = json.load(f)

lookup_coll.insert_many(file_data)

end = time.time()
print(round((end - start)/60 ,2))

0.52


# Updating documents

We decided to update docs inside *taxis_coll* collection in order to have payment and vendors name instead of ids.

In [11]:
# matching the docs
vendor1 = { "vendor_id" : 1}
# "values" to update
value_vend1 = { "$set": {"vendor_id":  "Creative Mobile Technologies"} }

vendor2 = { "vendor_id" : 2}
value_vend2 = { "$set": {"vendor_id":  "VeriFone Inc"} }

In [15]:
# updating documents
upd_vend1 = taxis_coll.update_many(vendor1, value_vend1)

print(upd_vend1.modified_count, "documents updated.")

3846105 documents updated.


In [14]:
# updating documents
upd_vend2 = taxis_coll.update_many(vendor2, value_vend2)

print(upd_vend2.modified_count, "documents updated.")

4913768 documents updated.


In [16]:
pay1 = {"pay_type" : 1}
pay2 = {"pay_type" : 2}
pay3 = {"pay_type" : 3}
pay4 = {"pay_type" : 4}
pay5 = {"pay_type" : 5}
pay6 = {"pay_type" : 6}

value_pay1 = {"$set" : {"pay_type":"CreditCard"}}
value_pay2 = {"$set" : {"pay_type":"Cash"}}
value_pay3 = {"$set" : {"pay_type":"NoCharge"}}
value_pay4 = {"$set" : {"pay_type":"NoCharge"}} 
value_pay5 = {"$set" : {"pay_type":"Unknown"}}
value_pay6 = {"$set" : {"pay_type":"VoidedTrip"}}

In [17]:
start = time.time()

upd_pay1 = taxis_coll.update_many(pay1, value_pay1)
upd_pay2 = taxis_coll.update_many(pay2, value_pay2)
upd_pay3 = taxis_coll.update_many(pay3, value_pay3)
upd_pay4 = taxis_coll.update_many(pay4, value_pay4)
upd_pay5 = taxis_coll.update_many(pay5, value_pay5)
upd_pay6 = taxis_coll.update_many(pay6, value_pay6)

end = time.time()
print(round((end - start)/60 ,2))

4.16


In [18]:
# checking that there are no more ids in vendor and payment field

for obj in taxis_coll.find().limit(1):
    pprint(obj)

{'_id': ObjectId('5ce04b71c25191cad58cc5ce'),
 'distance': 0.5,
 'do_loc': 24,
 'dropoff_date': '2018-01-01',
 'dropoff_time': '00:24:23',
 'fare': 4.5,
 'pass_count': 1,
 'pay_type': 'Cash',
 'pickup_date': '2018-01-01',
 'pickup_time': '00:21:05',
 'pu_loc': 41,
 'total': 5.8,
 'vendor_id': 'Creative Mobile Technologies'}


# Queries

#### 1) Number of trips per vendor_id

In [19]:
pip_vendor_trips = [
    {"$group": {"_id": "$vendor_id", "trips_count": { "$sum": 1}}},
]

trips_vendor = taxis_coll.aggregate(pip_vendor_trips)
trips_vendor = list(trips_vendor)
trips_vendor

[{'_id': 'VeriFone Inc', 'trips_count': 4913768},
 {'_id': 'Creative Mobile Technologies', 'trips_count': 3846106}]

#### 2) Most common payment_type

In [20]:
pip_common_pay = [
    {"$group": {"_id": "$pay_type", "num_trips" : { "$sum": 1}}},
    {"$sort": {"num_trips": -1}},
    {"$limit" : 1}
]

common_pay = taxis_coll.aggregate(pip_common_pay)
common_pay = list(common_pay)
common_pay

[{'_id': 'CreditCard', 'num_trips': 6105871}]

#### 3) Top 5 trips with distance > 3 AND amount < 15

In [21]:
top_five = taxis_coll.find({"distance"  : {"$gt" : 3}, "fare" : {"$lt" : 15}}).limit(5)
top_five = list(top_five)
top_five[0]

{'_id': ObjectId('5ce04b71c25191cad58cc5e2'),
 'vendor_id': 'VeriFone Inc',
 'pass_count': 5,
 'distance': 3.13,
 'pu_loc': 263,
 'do_loc': 143,
 'pay_type': 'Cash',
 'fare': 13.0,
 'total': 14.3,
 'pickup_date': '2018-01-01',
 'pickup_time': '00:25:19',
 'dropoff_date': '2018-01-01',
 'dropoff_time': '00:45:02'}

#### 4) Average distance per vendor

In [85]:
pip_avg_dist = [
    {"$group" : {"_id" : "$vendor_id", "avg" : {"$avg" : "$distance"}}},
    {"$project" : {"_id" : 1, "avg" : 1, "avg" : 1}}
]

avg_dist = taxis_coll.aggregate(pip_avg_dist)
avg_dist = list(avg_dist)
avg_dist

[{'_id': 'VeriFone Inc', 'avg': 2.910206959302922},
 {'_id': 'Creative Mobile Technologies', 'avg': 2.6683130418142404}]

#### 5) Trips with most common vendor AND NOT most common Borough

In [23]:
# retrieving Manhattan ids from lookup collection

manhattan_id = []
for el in lookup_coll.find({"Borough" : "Manhattan"}):
    manhattan_id.append(el['LocationID'])

len(manhattan_id)

69

In [25]:
pip_not = [
    {"$match" : {"vendor_id" : "Creative Mobile Technologies", "pu_loc" : {"$nin" :  manhattan_id}}},
    {"$project" : {"_id" : 0, "vendor_id" : 1, "pu_loc" : 1, "distance" : 1, "pay_type" : 1}}
]

not_most = taxis_coll.aggregate(pip_not)
not_most = list(not_most)
len(not_most)

369098

In [26]:
not_most[0]

{'vendor_id': 'Creative Mobile Technologies',
 'distance': 4.6,
 'pu_loc': 145,
 'pay_type': 'CreditCard'}

In [27]:
# checking whether we found the right docs or not

for obj in not_most:
    for el in manhattan_id: 
        if el == obj['pu_loc']:
            print('doh!')